### Imports

In [ ]:
from utils import *
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras import models, layers, optimizers, losses
from sklearn.metrics import classification_report

### Read/Preprocess Data

In [ ]:
IMG_SIZE=224
X, y = read_images()
X = resize_and_rescale(X, IMG_SIZE=IMG_SIZE, rescale=False)
X_train, y_train, X_val, y_val, X_test, y_test = split_and_shuffle(X, y)

# convert to rgb images
X_train = np.array([[[[x,x,x]  for x in x2] for x2 in x1] for x1 in X_train])
X_val = np.array([[[[x,x,x]  for x in x2] for x2 in x1] for x1 in X_val])
X_test = np.array([[[[x,x,x]  for x in x2] for x2 in x1] for x1 in X_test])

### Model

In [ ]:
def vgg16_model(learning_rate=0.0001):
    vgg16 = VGG16(include_top=False, weights="imagenet", input_shape = (IMG_SIZE,IMG_SIZE, 3))

    # prevent training of VGG16
    vgg16.trainable = False

    model = models.Sequential()
    model.add(vgg16)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(4096, activation="relu"))
    model.add(layers.Dense(4096, activation="relu"))
    model.add(layers.Dense(2, activation="sigmoid"))
    model.compile(optimizer=optimizers.Adam(learning_rate=learning_rate),
                  loss=losses.SparseCategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])
    return model

In [ ]:
model = vgg16_model()
model.summary()

### Validation and Testing

In [ ]:
epochs = 15

results = model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=epochs)

In [ ]:
plot_accuracy_comparison([results.history["accuracy"], results.history["val_accuracy"]],
                        "Training/Validation Accuracy Comparison",
                        ["Training Accuracy", "Validation Accuracy"])

In [ ]:
plot_loss_comparison([results.history["loss"], results.history["val_loss"]],
                     "Training/Validation Loss Comparison",
                     ["Training Loss", "Validation Loss"])

In [ ]:
model = vgg16_model()
results = model.fit(np.concatenate((X_train,X_val)), np.concatenate((y_train,y_val)), epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", acc)

# show confusion matrix
y_pred=np.argmax(model.predict(X_test), axis=-1)
plot_confusion_matrix(y_test, y_pred)

# show other metrics
print(classification_report(y_pred,y_test))

### Hyper-Parameter Selection

#### Epochs

In [ ]:
model = vgg16_model()
results = model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=30)

In [ ]:
plot_accuracy_comparison([results.history["accuracy"], results.history["val_accuracy"]],
                        "Training/Validation Accuracy Comparison",
                        ["Training Accuracy", "Validation Accuracy"])

In [ ]:
plot_loss_comparison([results.history["loss"], results.history["val_loss"]],
                     "Training/Validation Loss Comparison",
                     ["Training Loss", "Validation Loss"])

#### Learning Rate

In [ ]:
#https://machinelearningmastery.com/learning-rate-for-deep-learning-neural-networks/
learning_rates=[0.1, 0.01, 0.001, 0.0001, 0.00001]
acc = []
loss = []

for lr in learning_rates:
    model = vgg16_model(learning_rate=lr)
    results = model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=epochs)

    acc.append(results.history["val_accuracy"])
    loss.append(results.history["val_loss"])

In [ ]:
plot_accuracy_comparison(acc, "Learning Rate Comparison (Validation Accuracy)", learning_rates)

In [ ]:
plot_loss_comparison(loss, "Learning Rate Comparison (Validation Loss)", learning_rates)